In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [5]:
forbes2000 = pd.read_csv('../data/stock_market_data/forbes2000/csv/A.csv')
forbes2000.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,18-11-1999,28.612303,32.546494,62546380,35.765381,31.473534,27.066582
1,19-11-1999,28.478184,30.713518,15234146,30.758226,28.880545,24.836662
2,22-11-1999,28.657009,29.551144,6577870,31.473534,31.473534,27.066582
3,23-11-1999,28.612303,30.400572,5975611,31.205294,28.612303,24.605980
4,24-11-1999,28.612303,28.701717,4843231,29.998213,29.372318,25.259573


In [8]:
nyse = pd.read_csv('../data/stock_market_data/nyse/csv/AAC.csv')
nyse.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,25-03-2021,9.52,9.95,117100,9.950,9.87,9.87
1,26-03-2021,9.75,9.85,158300,9.960,9.96,9.96
2,29-03-2021,9.50,9.70,53600,9.902,9.78,9.78
3,30-03-2021,9.80,9.80,52300,9.850,9.83,9.83
4,31-03-2021,9.75,9.75,78600,9.900,9.90,9.90


In [9]:
sp500 = pd.read_csv('../data/stock_market_data/sp500/csv/A.csv')
sp500.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,18-11-1999,28.612303,32.546494,62546380,35.765381,31.473534,26.929762
1,19-11-1999,28.478184,30.713518,15234146,30.758226,28.880545,24.711119
2,22-11-1999,28.657009,29.551144,6577870,31.473534,31.473534,26.929762
3,23-11-1999,28.612303,30.400572,5975611,31.205294,28.612303,24.481607
4,24-11-1999,28.612303,28.701717,4843231,29.998213,29.372318,25.131897


In [10]:
all_data = pd.read_csv('../data/stock_market_data/all_data.csv', index_col=0)
all_data.head()

,Unnamed: 0,Date,Low,Open,Volume,High,Close,Adjusted Close,ticker
0,0,2005-01-03,16.809729,17.238913,3378826.0,17.296137,17.081545,14.689768,csv\A
1,1,2005-01-04,16.487841,17.010014,3746920.0,17.160229,16.630901,14.302225,csv\A
2,2,2005-01-05,16.537910,16.573677,3898603.0,16.917025,16.623749,14.296078,csv\A
3,3,2005-01-06,16.230330,16.738197,3158641.0,16.766809,16.258942,13.982349,csv\A
4,4,2005-01-07,16.187410,16.223175,2624326.0,16.416309,16.244635,13.970040,csv\A
